# 

In [180]:
import openai
from openai import OpenAI

# Point to the server
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

In [45]:
# Sentences to classify
sentences = [
    "I hate you and I hope you fail.",
    "What a beautiful day to go for a walk!",
    "Your idea is stupid and nobody cares."
]

sentences = [
    "#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon @USER URL",
    "#ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.",
]

# Build a single prompt
prompt = "Classify each of the following sentences as 'hate' or 'non-hate':\n\n"
for i, s in enumerate(sentences, 1):
    prompt += f"{i}. {s}\n"

prompt += "\nReturn the results in the format:\n<number>. <label>\n"

# Make one request for all sentences
response = client.completions.create(
    model="local model",  # currently unused
    prompt=prompt,
    max_tokens=50,
    temperature=0,
    stop=["Classify", "\n\n"]
)

# Print the raw model output
print(response.choices[0].text.strip())

1. hate
2. non-hate


In [181]:
from sklearn.metrics import classification_report, confusion_matrix

In [201]:
import pandas as pd

# evaluation data
# 15923,"#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon ⁦@USER URL",1

data = pd.read_csv("data/olid-test.csv")
data.head()

,id,text,labels
0,27014,"#ConstitutionDay is revered by Conservatives, ...",0
1,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
2,13876,#Watching #Boomer getting the news that she is...,0
3,60133,#NoPasaran: Unity demo to oppose the far-right...,1
4,83681,. . . What the fuck did he do this time?,1


In [211]:
def evaluate(prompt, data):
    
    # getting sentences from the datadrame along with their ground truth labels
    # sentences = []
    ground_truth = []
    
    # Build a single prompt
    prompt = "Classify each of the following lines as 'offensive' or 'non-offensive':\n\n"
    
    for id, row in data.iterrows():
        # sentences.append(row['text'])
        ground_truth.append(row['labels'])
        prompt += f"{id+1}. {row['text']}\n"

    # Do not Explain, Make sure that there are 10 labels, 1 label for each line, Just and ONLY in this format
    prompt += "\nReturn the results in this format:\n<number>. <label>\n"

    print(prompt)
    # Make one request for all sentences
    response = client.completions.create(
        model="local model",  # currently unused
        prompt=prompt,
        max_tokens=200,
        temperature=0,
        stop=["Classify", "\n\n"]
    )

    # Processing the model output
    predicted = response.choices[0].text.strip().split("\n")
    final_predictions = []
    
    print(predicted)
    for p in predicted:
        try:
            p = p.split(". ")[1]
        except:
            print(p)
            continue
        
        if p not in ['offensive', 'non-offensive']:
            raise ValueError("Unexpected label: " + p)

        final_predictions.append(1 if p == 'offensive' else 0)

    return final_predictions, ground_truth

In [212]:
# get 10:20 from data
data[10:20]

,id,text,labels
10,22452,𝓒-𝓒𝓮𝓭𝓻𝓪𝓷.. The Nord cannot make a single move...,0
11,15565,#ConsTOO THE PLACE FOR FED UP CONSERVATIVES !!!,0
12,64376,#GreatAwakening #QAnon #PatriotsUnited #WWG1WG...,0
13,12588,@USER Nigga ware da hits at,1
14,34263,#StopKavanaugh he is liar like the rest of the...,1
15,65773,#Dayspromo week of September 17th #days #dool ...,0
16,95457,#BiggBossTelugu2 Let’s all include @USER in al...,0
17,24930,#Dallas#Policewoman kills neighbor in HIS own ...,0
18,15938,Always smack URL,0
19,45712,"#BREAKING: After a week-long trial, a Linn Cou...",0


In [213]:
# Prompts

prompts = {
    "prompt1": "Classify each of the following sentences as 'offensive' or 'non-offensive':\n\n",
    "prompt2": """You are a classifier of hate speech. The following data contains language that is either non offensive or offensive. 
Please note these are the definitions of offensive/non-offensive so try to stick to these guidelines.
'offensive': A post that contains insults, threats, or profanity directed towards a person or group.
'non-offensive': A post that is neutral, polite, or does not contain any harmful language."""
,
    "prompt3": "",
}

all_predictions = []
all_truth_labels = []
for piece in range(0, 860, 10):
    pred, truth = evaluate(prompts["prompt1"], data[piece:piece+10])
    
    print(pred)
    
    all_predictions.extend(pred)
    all_truth_labels.extend(truth) 

Classify each of the following lines as 'offensive' or 'non-offensive':

1. #ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.
2. #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendment #RNC #USMC #veterans @USER    @USER @USER @USER @USER   #fakereporting #THESWAMP #dnc #liberals @USER @USER #fakeoutrage @USER  First, it reduces the ca URL
3. #Watching #Boomer getting the news that she is still up for parole always makes me smile. #Wentworth Finale...@USER is such a treasure. URL
4. #NoPasaran: Unity demo to oppose the far-right in #London – #antifa #Oct13 — Enough is Enough! URL
5. . . . What the fuck did he do this time?
6. #RAP is a form of ART! Used to express yourself freely. It does not gv the green light or excuse the behavior of acting like an animal! She is not in the streets of the BX where violence is a way of living. Elevate yourself boo and get on @USER level for longevity! #QUEEN👑
7. @USER Do you get the feeling he is 

ValueError: Unexpected label: This answer provides one possible classification for each line.

In [ ]:
print(classification_report(, , target_names=['non-hate', 'hate']))

              precision    recall  f1-score   support

    non-hate       1.00      0.83      0.91         6
        hate       0.80      1.00      0.89         4

    accuracy                           0.90        10
   macro avg       0.90      0.92      0.90        10
weighted avg       0.92      0.90      0.90        10

